<a href="https://colab.research.google.com/github/biedun/JEPSis/blob/main/my_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nibabel as nib
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from skimage.transform import rotate
from skimage.util import montage
import os
from sklearn.model_selection import train_test_split
import keras
import cv2
import tensorflow
import random
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import numpy as np
from keras.callbacks import CSVLogger
import keras.backend as K
import zipfile
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df18 = pd.read_csv("/content/drive/MyDrive/Colab/JEPSis/Tumor_Data/2018/survival_data.csv")

In [ ]:
df18

,BraTS18ID,Age,Survival,ResectionStatus
0,Brats18_TCIA08_167_1,74.907,153,NaN
1,Brats18_TCIA08_242_1,66.479,147,NaN
2,Brats18_TCIA08_319_1,64.860,254,NaN
3,Brats18_TCIA08_469_1,63.899,519,NaN
4,Brats18_TCIA08_218_1,57.345,346,NaN
...,...,...,...,...
158,Brats18_CBICA_ABB_1,68.493,465,GTR
159,Brats18_CBICA_AAP_1,39.068,788,GTR
160,Brats18_CBICA_AAL_1,54.301,464,GTR
161,Brats18_CBICA_AAG_1,52.263,616,GTR


In [ ]:
df19 = pd.read_csv("/content/drive/MyDrive/Colab/JEPSis/Tumor_Data/2019/survival_data_2019.csv")

In [ ]:
df20 = pd.read_csv("/content/drive/MyDrive/Colab/JEPSis/Tumor_Data/MICCAI_BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/survival_info_2020.csv")

In [ ]:
df20_map = pd.read_csv("/content/drive/MyDrive/Colab/JEPSis/Tumor_Data/MICCAI_BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/name_mapping.csv")

In [ ]:
df20_map

,Grade,BraTS_2017_subject_ID,BraTS_2018_subject_ID,TCGA_TCIA_subject_ID,BraTS_2019_subject_ID,BraTS_2020_subject_ID
0,HGG,Brats17_CBICA_AAB_1,Brats18_CBICA_AAB_1,NaN,BraTS19_CBICA_AAB_1,BraTS20_Training_001
1,HGG,Brats17_CBICA_AAG_1,Brats18_CBICA_AAG_1,NaN,BraTS19_CBICA_AAG_1,BraTS20_Training_002
2,HGG,Brats17_CBICA_AAL_1,Brats18_CBICA_AAL_1,NaN,BraTS19_CBICA_AAL_1,BraTS20_Training_003
3,HGG,Brats17_CBICA_AAP_1,Brats18_CBICA_AAP_1,NaN,BraTS19_CBICA_AAP_1,BraTS20_Training_004
4,HGG,Brats17_CBICA_ABB_1,Brats18_CBICA_ABB_1,NaN,BraTS19_CBICA_ABB_1,BraTS20_Training_005
...,...,...,...,...,...,...
364,HGG,NaN,NaN,NaN,NaN,BraTS20_Training_365
365,HGG,NaN,NaN,NaN,NaN,BraTS20_Training_366
366,HGG,NaN,NaN,NaN,NaN,BraTS20_Training_367
367,HGG,NaN,NaN,NaN,NaN,BraTS20_Training_368


In [ ]:
df19_map = pd.read_csv("/content/drive/MyDrive/Colab/JEPSis/Tumor_Data/2019/name_mapping.csv")

In [ ]:
df19_map

,Grade,BraTS_2017_subject_ID,BraTS_2018_subject_ID,TCGA_TCIA_subject_ID,BraTS_2019_subject_ID
0,HGG,Brats17_CBICA_AAB_1,Brats18_CBICA_AAB_1,NaN,BraTS19_CBICA_AAB_1
1,HGG,Brats17_CBICA_AAG_1,Brats18_CBICA_AAG_1,NaN,BraTS19_CBICA_AAG_1
2,HGG,Brats17_CBICA_AAL_1,Brats18_CBICA_AAL_1,NaN,BraTS19_CBICA_AAL_1
3,HGG,Brats17_CBICA_AAP_1,Brats18_CBICA_AAP_1,NaN,BraTS19_CBICA_AAP_1
4,HGG,Brats17_CBICA_ABB_1,Brats18_CBICA_ABB_1,NaN,BraTS19_CBICA_ABB_1
...,...,...,...,...,...
330,LGG,Brats17_TCIA_621_1,Brats18_TCIA13_621_1,TCGA-HT-8018,BraTS19_TCIA13_621_1
331,LGG,Brats17_TCIA_653_1,Brats18_TCIA13_653_1,TCGA-HT-8111,BraTS19_TCIA13_653_1
332,LGG,Brats17_TCIA_642_1,Brats18_TCIA13_642_1,TCGA-HT-8114,BraTS19_TCIA13_642_1
333,LGG,Brats17_TCIA_624_1,Brats18_TCIA13_624_1,TCGA-HT-8563,BraTS19_TCIA13_624_1


In [ ]:
# Merging
merged = pd.merge(df19_map, df20_map[['Grade']], on='BraTS_2020_subject_ID', how='left')
# Renaming the columns for clarity
'''geo_merged.rename(columns={'Airport Name': 'departure_airport_name',
                               'City': 'departure_city',
                               'Country': 'departure_country',
                               'Latitude': 'departure_latitude',
                               'Longitude': 'departure_longitude',
                               'Altitude': 'departure_altitude'}, inplace=True)
# Dropping the redundant IATA Code column
geo_merged.drop(columns=['IATA Code'], inplace=True)
# Displaying the updated dataframe
geo_merged.head(10)'''

KeyError: 'BraTS_2020_subject_ID'

In [ ]:
!pip install pivottablejs
from pivottablejs import pivot_ui
pivot_ui(df19_map)

In [ ]:
pivot_ui(df19_map)

In [ ]:
# Unzip dataset
path_to_zip_file = "/content/drive/MyDrive/Colab/JEPSis/Tumor_Data/MICCAI_BraTS2020_TrainingData.zip"
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall("/content/drive/MyDrive/Colab/JEPSis/Tumor_Data/MICCAI_BraTS2020_TrainingData_Pawel")

In [ ]:
# Import OS for navigation and environment set up
import os
# Check current location, '/content' is the Colab virtual machine
os.getcwd()
# Enable the Kaggle environment, use the path to the directory your Kaggle API JSON is stored in
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/Colab/kaggle'

In [ ]:
!pip install kaggle

In [ ]:
! ls -a /content/drive/MyDrive/Colab/kaggle

kaggle.json


In [ ]:
# Navigate into Drive where you want to store your Kaggle data
os.chdir('/content/drive/MyDrive/Colab/kaggle')
# Paste and run the copied API command, the data will download to the current directory
!kaggle datasets download -d andrewmvd/brain-tumor-segmentation-in-mri-brats-2015
# Check contents of directory, you should see the .zip file for the competition in your Drive
os.listdir()


100% 7.81G/7.82G [02:48<00:00, 73.6MB/s]
100% 7.82G/7.82G [02:48<00:00, 49.8MB/s]


['kaggle.json', 'brain-tumor-segmentation-in-mri-brats-2015.zip']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Complete path to storage location of the .zip file of data
zip_path = '/content/drive/MyDrive/Colab/kaggle/brain-tumor-segmentation-in-mri-brats-2015.zip'
# Check current directory (be sure you're in the directory where Colab operates: '/content')
os.getcwd()
# Copy the .zip file into the present directory
!cp '{zip_path}' .
# Unzip quietly
!unzip -q '/content/drive/MyDrive/Colab/kaggle/brain-tumor-segmentation-in-mri-brats-2015.zip'
# View the unzipped contents in the virtual machine
#os.listdir()